In [1]:
import pickle

with open('original_model_weights.pkl', 'rb') as file:
    original_model_weights = pickle.load(file)

original_model_weights



[array([[ 2.5940590e-02, -4.0624756e-02, -4.1460260e-03, ...,
         -4.8734844e-02,  1.9328283e-02, -4.8259225e-02],
        [-3.1265214e-02,  8.2980007e-02,  4.3803539e-02, ...,
          3.1056687e-02,  1.7932033e-02,  6.9596302e-03],
        [ 4.3348661e-03, -5.9631437e-02, -1.1196332e-01, ...,
          9.3319051e-02,  5.8863158e-03,  2.1785971e-02],
        ...,
        [ 3.3533853e-02,  6.6992859e-03, -6.3023791e-03, ...,
          2.3112213e-02,  4.6645924e-02, -7.4068695e-02],
        [ 7.1984477e-02, -3.1879887e-02, -2.1005819e-02, ...,
          8.7428585e-02, -3.9414110e-05,  5.2459769e-02],
        [-1.4843541e-02,  1.2660646e-02,  6.5805893e-03, ...,
          9.9764671e-03, -1.6177995e-02, -7.8036278e-02]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.

In [8]:

with open('compressed_model_weights.pkl', 'rb') as file:
    compressed_model_weights = pickle.load(file)

# compressed_model_weights


In [9]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import CompressibleNN


kernel_initializer = 'he_normal'
activation = "relu"


def get_model(chs=128):
    shape = (28, 28, 1)

    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs * 2, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model
    

model = get_model()

compNN = CompressibleNN(model)
decompressed_weights = compNN.decompressNN(compressed_model_weights)

print(compressed_weights)
print(decompressed_weights)

# Compare the original weights with the decompressed weights
differences = compNN.compare_weights(original_model_weights, decompressed_weights)

print("Differences between original and decompressed weights:")
for i, diff in enumerate(differences):
    print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")
    

IndexError: list index out of range

In [169]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman


class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        self.codec = []

    def decompressNN(self, compressed_weights):
        # Decompress the weights using Huffman coding
        decompressed_weights = []
        for i, compressed_data in enumerate(compressed_weights):
            decoder = self.codec[i]
            decompressed_data = decoder.decode(compressed_data)  # Use decode() directly

            print("Decompressed data size:", len(decompressed_data))
            weight_shape = self.net_model.get_weights()[i].shape  # Retrieve the shape of the corresponding weight tensor
            decompressed_size = np.prod(weight_shape) * np.dtype(np.float32).itemsize

            print("Expected decompressed size:", decompressed_size)
            print("Actual decompressed size:", len(decompressed_data))

            if len(decompressed_data) < decompressed_size:
                decompressed_data += b'\x00' * (decompressed_size - len(decompressed_data))

            elif len(decompressed_data) > decompressed_size:
                decompressed_data = decompressed_data[:decompressed_size]

            decompressed_array = np.frombuffer(bytes(decompressed_data), dtype=np.float32)
            decompressed_weights.append(decompressed_array.reshape(weight_shape))

        return decompressed_weights

    def call(self, inputs):
        return self.net_model(inputs)

    def reshape_weights(self, inputs):
        reshaped_weights = []
        current_index = 0
        for weight_tensor in self.net_model.get_weights():
            weight_shape = weight_tensor.shape
            weight_size = np.prod(weight_shape)
            # Reshape the weight tensor based on the size of the input array
            reshaped_weights.append(weight_tensor.reshape(weight_shape))
            current_index += weight_size
        return reshaped_weights
    
    def compare_weights(self, original_weights, decompressed_weights):
        differences = []
        for orig, decomp in zip(original_weights, decompressed_weights):
            orig_shape = orig.shape
            decomp_shape = decomp.shape
            decomp_reshaped = decomp.reshape(orig_shape) if orig_shape == decomp_shape else decomp
            diff = np.abs(orig - decomp_reshaped)
            differences.append(diff)
        return differences


    

model = get_model()
compNN = CompressibleNN(model)

# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42
inputs = random_state.rand(28, 28, 1)  # Generate random input data

# Compress the weights
compressed_weights = compNN.compressNN(inputs)

# Decompress the weights
decompressed_weights = compNN.decompressNN(compressed_weights)

# print(compressed_weights)
# print(decompressed_weights)

# Compare the original weights with the decompressed weights
differences = compNN.compare_weights(model.get_weights(), decompressed_weights)
print("Differences between original and decompressed weights:")
for i, diff in enumerate(differences):
    print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")
    
    
    

6
Weight tensor shape: (784, 256)
Weight tensor size: 200704
Weight flattened size: 200704
Compressed data size: 739794
Weight tensor shape: (256,)
Weight tensor size: 256
Weight flattened size: 256
Compressed data size: 128
Weight tensor shape: (256, 128)
Weight tensor size: 32768
Weight flattened size: 32768
Compressed data size: 121327
Weight tensor shape: (128,)
Weight tensor size: 128
Weight flattened size: 128
Compressed data size: 64
Weight tensor shape: (128, 10)
Weight tensor size: 1280
Weight flattened size: 1280
Compressed data size: 4701
Weight tensor shape: (10,)
Weight tensor size: 10
Weight flattened size: 10
Compressed data size: 5
Decompressed data size: 802816
Expected decompressed size: 802816
Actual decompressed size: 802816
Decompressed data size: 1024
Expected decompressed size: 1024
Actual decompressed size: 1024
Decompressed data size: 131072
Expected decompressed size: 131072
Actual decompressed size: 131072
Decompressed data size: 512
Expected decompressed siz

In [170]:
print(model.get_weights())

[array([[-0.0387225 , -0.08137392, -0.02914773, ..., -0.00919168,
         0.02946452,  0.04639422],
       [-0.05397551, -0.00483679, -0.04930591, ...,  0.07424383,
         0.0271853 , -0.01563342],
       [-0.00271451, -0.0424049 , -0.00124385, ...,  0.00815904,
        -0.0186043 , -0.02493658],
       ...,
       [ 0.06878325,  0.03475486,  0.06290235, ...,  0.04817623,
        -0.08055975,  0.05437632],
       [-0.07352863,  0.01516827,  0.10736844, ..., -0.01694267,
         0.06866149,  0.00142415],
       [-0.03066814,  0.03886299,  0.03796422, ...,  0.08156086,
         0.02407725, -0.06137323]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [171]:
print(decompressed_weights)

[array([[-0.0387225 , -0.08137392, -0.02914773, ..., -0.00919168,
         0.02946452,  0.04639422],
       [-0.05397551, -0.00483679, -0.04930591, ...,  0.07424383,
         0.0271853 , -0.01563342],
       [-0.00271451, -0.0424049 , -0.00124385, ...,  0.00815904,
        -0.0186043 , -0.02493658],
       ...,
       [ 0.06878325,  0.03475486,  0.06290235, ...,  0.04817623,
        -0.08055975,  0.05437632],
       [-0.07352863,  0.01516827,  0.10736844, ..., -0.01694267,
         0.06866149,  0.00142415],
       [-0.03066814,  0.03886299,  0.03796422, ...,  0.08156086,
         0.02407725, -0.06137323]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
Here is my neural network model like below

"
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
    
kernel_initializer = 'he_normal'
activation = "relu"

def get_model(chs=128):
    shape=(28,28,1)
    
    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs*2, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model

    def compressNN(self, inputs):
            # implement 

    def decompressNN(self, inputs):
            # implement 

    def call(self, inputs):
        return self.net_model(inputs)
    
    def compare_weights(self, original_weights, decompressed_weights):
            # implement
    
model = get_model()
compNN = CompressibleNN(model)

# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42


# Compress the weights
compressed_weights = compNN.compressNN(inputs)

# Decompress the weights
decompressed_weights = compNN.decompressNN(compressed_weights)
"


I have a network model. I want compress it and decompress it. Put the weights back into the neural network. 
And then see the after compress and decompress and then match with the original.

Can you implement function compressNN, decompressNN and ?

SyntaxError: unterminated string literal (detected at line 3) (190610571.py, line 3)

In [88]:
import joblib

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        self.compressor = dahuffman.HuffmanCodec.from_data([])  # Initialize an empty compressor
        
    def compressNN(self, inputs):
        # Get the current weights of the network
        weights = self.net_model.get_weights()

        # Serialize the weights using joblib
        with open("weights.pkl", "wb") as file:
            joblib.dump(weights, file)

        # Read the serialized weights
        with open("weights.pkl", "rb") as file:
            serialized_weights = file.read()

        # Compress the serialized weights using Huffman encoding
        encoder = dahuffman.HuffmanCodec.from_data(serialized_weights)
        compressed_weights = encoder.encode(serialized_weights)

        return compressed_weights

    def decompressNN(self, compressed_weights):
        # Decode the compressed weights using Huffman decoding
        decoder = dahuffman.HuffmanCodec.from_data(compressed_weights)
        serialized_weights = decoder.decode(compressed_weights)

        # Deserialize the weights using joblib
        with open("decompressed_weights.pkl", "wb") as file:
            file.write(serialized_weights)

        # Load the deserialized weights
        with open("decompressed_weights.pkl", "rb") as file:
            weights = joblib.load(file)

        # Set the decompressed weights to the network
        self.net_model.set_weights(weights)

        return self.net_model


# Instantiate the CompressibleNN class and create the original model
model = get_model()  # Replace with your own model creation code

# Save the original model's architecture
original_architecture = model.to_json()

compNN = CompressibleNN(model)

# Generate some random data for testing
random_state = np.random.RandomState(42)  # Set the seed value to 42
inputs = np.random.rand(10, 28, 28, 1)

# Compress the weights
compressed_weights = compNN.compressNN(inputs)

# Decompress the weights
decompressed_weights = compNN.decompressNN(compressed_weights)

# Compare the original weights with the decompressed weights
# differences = compNN.compare_weights(original_weights, decompressed_weights)

# # Print the maximum difference for each weight matrix

KeyError: 51